- urllib
    - get 请求
    - post 请求
    - Handler 处理器和自定义 Opener
    - URLError 与 HTTPError


- requests
    - get 请求
    - post 请求
    - 文件上传
    - 获取 Cookie
    - 证书验证

## 2.1 urllib

In [1]:
from urllib import request

In [4]:
with request.urlopen('https://api.douban.com/v2/book/2224879') as f:
    data = f.read()
    print('Status: {0} {1}'.format(f.status, f.reason))
    for k, v in f.getheaders():
        print('{0}: {1}'.format(k, v))
    print('Data:', data.decode('utf-8'))

Status: 200 OK
Date: Thu, 01 Nov 2018 07:50:06 GMT
Content-Type: application/json; charset=utf-8
Content-Length: 9268
Connection: close
Vary: Accept-Encoding
X-Ratelimit-Remaining2: 96
X-Ratelimit-Limit2: 100
Expires: Sun, 1 Jan 2006 01:00:00 GMT
Pragma: no-cache
Cache-Control: must-revalidate, no-cache, private
Set-Cookie: bid=pFQtRQ_3D3k; Expires=Fri, 01-Nov-19 07:50:06 GMT; Domain=.douban.com; Path=/
X-DOUBAN-NEWBID: pFQtRQ_3D3k
X-DAE-Node: anson60
X-DAE-App: book
Server: dae
X-Frame-Options: SAMEORIGIN
Data: {"rating":{"max":10,"numRaters":1092,"average":"8.2","min":0},"subtitle":"","pubdate":"1977年4月","image":"https://img3.doubanio.com\/view\/subject\/m\/public\/s3996235.jpg","binding":"","images":{"small":"https://img3.doubanio.com\/view\/subject\/s\/public\/s3996235.jpg","large":"https://img3.doubanio.com\/view\/subject\/l\/public\/s3996235.jpg","medium":"https://img3.doubanio.com\/view\/subject\/m\/public\/s3996235.jpg"},"alt":"https:\/\/book.douban.com\/subject\/1916533\/","id